# **구글 드라이브 연결**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/집중교육/팀프로젝트/집교1팀플/top5 accuarcy

/content/drive/.shortcut-targets-by-id/1R8w9Au88wZukHx9BnC0_U6CoaYSOY2IO/집교1팀플/top5 accuarcy


In [3]:
%ls

combined_model.pth                 get_5_prediction.ipynb  rnn_model.pth    user_img_new.txt
get_5_prediction_googleData.ipynb  resnet101_64_3000.pth   Untitled0.ipynb  user_img.txt


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchsummary import summary
import numpy as np
import random
import time
import pickle
from torch.nn.utils.rnn import pad_sequence
import os
import pandas as pd
import json
import torch.nn.functional as F


random.seed(10)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# **데이터 처리**

In [38]:
model_path = './resnet101_64_3000.pth'
rnnmodel_path = './rnn_model.pth'
combined_model_path = './combined_model.pth'

data_path = '../데이터및라벨/penPositions.pkl'
label_path = '../데이터및라벨/labels.pkl'

In [23]:
_label = ['The Eiffel Tower', 'The Great Wall of China', 'The Mona Lisa', 'aircraft carrier', 'airplane', 'alarm clock', 'ambulance', 'angel', 'animal migration', 'ant', 'anvil', 'apple', 'arm', 'asparagus', 'axe', 'backpack', 'banana', 'bandage', 'barn', 'baseball bat', 'baseball', 'basket', 'basketball', 'bat', 'bathtub', 'beach', 'bear', 'beard', 'bed', 'bee', 'belt', 'bench', 'bicycle', 'binoculars', 'bird', 'birthday cake', 'blackberry', 'blueberry', 'book', 'boomerang', 'bottlecap', 'bowtie', 'bracelet', 'brain', 'bread', 'bridge', 'broccoli', 'broom', 'bucket', 'bulldozer', 'bus', 'bush', 'butterfly', 'cactus', 'cake', 'calculator', 'calendar', 'camel', 'camera', 'camouflage', 'campfire', 'candle', 'cannon', 'canoe', 'car', 'carrot', 'castle', 'cat', 'ceiling fan', 'cell phone', 'cello', 'chair', 'chandelier', 'church', 'circle', 'clarinet', 'clock', 'cloud', 'coffee cup', 'compass', 'computer', 'cookie', 'cooler', 'couch', 'cow', 'crab', 'crayon', 'crocodile', 'crown', 'cruise ship', 'cup', 'diamond', 'dishwasher', 'diving board', 'dog', 'dolphin', 'donut', 'door', 'dragon', 'dresser', 'drill', 'drums', 'duck', 'dumbbell', 'ear', 'elbow', 'elephant', 'envelope', 'eraser', 'eye', 'eyeglasses', 'face', 'fan', 'feather', 'fence', 'finger', 'fire hydrant', 'fireplace', 'firetruck', 'fish', 'flamingo', 'flashlight', 'flip flops', 'floor lamp', 'flower', 'flying saucer', 'foot', 'fork', 'frog', 'frying pan', 'garden hose', 'garden', 'giraffe', 'goatee', 'golf club', 'grapes', 'grass', 'guitar', 'hamburger', 'hammer', 'hand', 'harp', 'hat', 'headphones', 'hedgehog', 'helicopter', 'helmet', 'hexagon', 'hockey puck', 'hockey stick', 'horse', 'hospital', 'hot air balloon', 'hot dog', 'hot tub', 'hourglass', 'house plant', 'house', 'hurricane', 'ice cream', 'jacket', 'jail', 'kangaroo', 'key', 'keyboard', 'knee', 'knife', 'ladder', 'lantern', 'laptop', 'leaf', 'leg', 'light bulb', 'lighter', 'lighthouse', 'lightning', 'line', 'lion', 'lipstick', 'lobster', 'lollipop', 'mailbox', 'map', 'marker', 'matches', 'megaphone', 'mermaid', 'microphone', 'microwave', 'monkey', 'moon', 'mosquito', 'motorbike', 'mountain', 'mouse', 'moustache', 'mouth', 'mug', 'mushroom', 'nail', 'necklace', 'nose', 'ocean', 'octagon', 'octopus', 'onion', 'oven', 'owl', 'paint can', 'paintbrush', 'palm tree', 'panda', 'pants', 'paper clip', 'parachute', 'parrot', 'passport', 'peanut', 'pear', 'peas', 'pencil', 'penguin', 'piano', 'pickup truck', 'picture frame', 'pig', 'pillow', 'pineapple', 'pizza', 'pliers', 'police car', 'pond', 'pool', 'popsicle', 'postcard', 'potato', 'power outlet', 'purse', 'rabbit', 'raccoon', 'radio', 'rain', 'rainbow', 'rake', 'remote control', 'rhinoceros', 'rifle', 'river', 'roller coaster', 'rollerskates', 'sailboat', 'sandwich', 'saw', 'saxophone', 'school bus', 'scissors', 'scorpion', 'screwdriver', 'sea turtle', 'see saw', 'shark', 'sheep', 'shoe', 'shorts', 'shovel', 'sink', 'skateboard', 'skull', 'skyscraper', 'sleeping bag', 'smiley face', 'snail', 'snake', 'snorkel', 'snowflake', 'snowman', 'soccer ball', 'sock', 'speedboat', 'spider', 'spoon', 'spreadsheet', 'square', 'squiggle', 'squirrel', 'stairs', 'star', 'steak', 'stereo', 'stethoscope', 'stitches', 'stop sign', 'stove', 'strawberry', 'streetlight', 'string bean', 'submarine', 'suitcase', 'sun', 'swan', 'sweater', 'swing set', 'sword', 'syringe', 't-shirt', 'table', 'teapot', 'teddy-bear', 'telephone', 'television', 'tennis racquet', 'tent', 'tiger', 'toaster', 'toe', 'toilet', 'tooth', 'toothbrush', 'toothpaste', 'tornado', 'tractor', 'traffic light', 'train', 'tree', 'triangle', 'trombone', 'truck', 'trumpet', 'umbrella', 'underwear', 'van', 'vase', 'violin', 'washing machine', 'watermelon', 'waterslide', 'whale', 'wheel', 'windmill', 'wine bottle', 'wine glass', 'wristwatch', 'yoga', 'zebra', 'zigzag']

In [24]:
def penPosToSequence(penPos):
    sequence = []
    for stroke in range(len(penPos)):
      x = penPos[stroke][0]
      y = penPos[stroke][1]
      for i in range(len(x)):
        if i == 0:
          penState = 0
        elif i == len(x) - 1:
          penState = 2
        else:
          penState = 1
        sequence.append([x[i], y[i], penState])
    return sequence

In [25]:
# Bresenham's line algorithm을 이용하여 두 좌표를 연결하는 함수
def draw_line(x0, y0, x1, y1, matrix):
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy

    while True:
        if 0 <= x0 < 64 and 0 <= y0 < 64:  # 좌표가 행렬 범위 내에 있을 때만 적용
            matrix[y0, x0] = 1  # 좌표를 1로 설정
        if x0 == x1 and y0 == y1:
            break
        e2 = err * 2
        if e2 > -dy:
            err -= dy
            x0 += sx
        if e2 < dx:
            err += dx
            y0 += sy

# stroke_array를 받아서 CNN에 사용될 데이터를 반환하는 함수
def process_stroke_array(stroke_array):
    # CNN에 사용할 데이터 초기화
    cnn_data = np.zeros((1, 64, 64), dtype=np.uint8)

    # 64x64 빈 행렬 생성
    matrix = np.zeros((64, 64), dtype=np.uint8)

    # 각 좌표 간에 선을 그림
    for stroke in stroke_array:
        x_coords = stroke[0]
        y_coords = stroke[1]

        # 좌표 값 조정
        x_coords = [int(x / 4) for x in x_coords]
        y_coords = [int(y / 4) for y in y_coords]

        # Bresenham's line algorithm으로 좌표 간 선 그리기
        for i in range(len(x_coords) - 1):
            x0, y0 = x_coords[i], y_coords[i]
            x1, y1 = x_coords[i + 1], y_coords[i + 1]
            draw_line(x0, y0, x1, y1, matrix)

    # 결과를 CNN 데이터에 추가
    cnn_data[0] = matrix

    return cnn_data

In [26]:
with open(data_path, 'rb') as f:
    penPositions = pickle.load(f)

with open(label_path, 'rb') as f:
    labels = pickle.load(f)

In [27]:
class CombinedPenDataset(Dataset):
    def __init__(self, pen_positions, labels):
        self.pen_positions = pen_positions
        self.labels = labels

    def __len__(self):
        return len(self.pen_positions)

    def __getitem__(self, idx):
        pen_position = self.pen_positions[idx]

        sequence = penPosToSequence(pen_position)  # 시퀀스 생성
        sequence_tensor = torch.tensor(sequence, dtype=torch.float32)

        stroke_array = process_stroke_array(pen_position)  # CNN에 입력될 64x64 이미지
        stroke_tensor = torch.tensor(stroke_array, dtype=torch.float32)

        label = self.labels[idx]
        label_tensor = torch.tensor(label, dtype=torch.long)

        return stroke_tensor, sequence_tensor, label_tensor

In [28]:
# 새로운 CombinedPenDataset 사용
combined_dataset = CombinedPenDataset(penPositions, labels)

# 데이터셋 크기 분할 (70% 훈련, 20% 검증, 10% 테스트)
train_size = int(0.7 * len(combined_dataset))
val_size = int(0.2 * len(combined_dataset))
test_size = len(combined_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(combined_dataset, [train_size, val_size, test_size])

In [29]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # 배치의 각 요소를 cnn_data, sequence, label로 분리
    cnn_data, sequences, labels = zip(*batch)

    # CNN 데이터는 텐서로 스택
    cnn_data = torch.stack(cnn_data)

    # RNN 시퀀스에 대해 패딩 적용
    sequences_padded = pad_sequence([torch.tensor(seq, dtype=torch.float32) for seq in sequences],
                                    batch_first=True, padding_value=0)

    # 레이블을 텐서로 변환
    labels = torch.tensor(labels)

    return cnn_data, sequences_padded, labels


In [30]:
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

# **이미지 확인**

In [31]:
# import matplotlib.pyplot as plt

# plt.imshow(cnn_data[0, 0], cmap='gray')
# plt.title(labels[image_index])
# plt.show()

# **모델**

In [32]:
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size = 3, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size = 1, bias = False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.relu = nn.ReLU()
        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, stride = stride, kernel_size = 1, bias = False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion)
            )

    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=345):
        super().__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, 345)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def getResnetModel():
    return ResNet(BottleNeck, [2, 3, 22, 2])

In [33]:
class HParams():
    def __init__(self):
        self.enc_hidden_size = 2048
        self.Nz = 1024
        self.num_classes = 345
        self.overfitting_count = 10
        self.lr = 0.001
        self.epochs = 30

        self.dropout = 0.9


hp = HParams()

class EncoderRNN(nn.Module):
    def __init__(self):
        super(EncoderRNN, self).__init__()
        self.lstm = nn.LSTM(3, hp.enc_hidden_size, dropout = hp.dropout, bidirectional=True, batch_first=True)  # 입력 크기 3으로 설정
        self.fc_mu = nn.Linear(2 * hp.enc_hidden_size, hp.Nz)
        self.fc_sigma = nn.Linear(2 * hp.enc_hidden_size, hp.Nz)

    def forward(self, inputs, batch_size, hidden_cell=None):
        if hidden_cell is None:
            hidden = torch.zeros(2, batch_size, hp.enc_hidden_size).cuda()
            cell = torch.zeros(2, batch_size, hp.enc_hidden_size).cuda()
            hidden_cell = (hidden, cell)

        # LSTM에 시퀀스를 입력
        _, (hidden, _) = self.lstm(inputs.float(), hidden_cell)

        # 양방향 LSTM의 출력을 결합
        hidden_forward, hidden_backward = torch.split(hidden, 1, 0)
        hidden_cat = torch.cat([hidden_forward.squeeze(0), hidden_backward.squeeze(0)], 1)

        # mu와 sigma를 계산하여 잠재 벡터 z 생성
        mu = self.fc_mu(hidden_cat)
        sigma_hat = self.fc_sigma(hidden_cat)
        sigma = torch.exp(sigma_hat / 2.)
        z = mu + sigma * torch.randn_like(mu)

        return z, mu, sigma_hat


class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.encoder = EncoderRNN()  # 인코더 (시퀀스 데이터를 인코딩)
        self.fc = nn.Linear(hp.Nz, num_classes)  # 잠재 벡터 z를 분류하기 위한 fully connected layer

    def forward(self, inputs):
        batch_size = inputs.size(0)
        z, mu, sigma_hat = self.encoder(inputs, batch_size)  # 인코더 호출, z는 잠재 벡터
        logits = self.fc(z)  # 잠재 벡터 z를 분류기 입력으로 사용
        return logits





In [34]:
import torch.nn.functional as F

class CombinedModel(nn.Module):
    def __init__(self, cnn_model, rnn_model, num_classes):
        super(CombinedModel, self).__init__()
        self.cnn_model = cnn_model
        self.rnn_model = rnn_model
        self.fc = nn.Linear(345 * 2, num_classes)

        for param in self.cnn_model.parameters():
            param.requires_grad = False

        for param in self.rnn_model.parameters():
            param.requires_grad = False


    def forward(self, cnn_data, rnn_data):
        cnn_output = self.cnn_model(cnn_data)
        # cnn_probs = F.softmax(cnn_output, dim=1)

        rnn_output = self.rnn_model(rnn_data)
        # rnn_probs = F.softmax(rnn_output, dim=1)

        combined_output = torch.cat((cnn_output, rnn_output), dim=1)

        final_output = self.fc(combined_output)

        # final_probs = F.softmax(final_output, dim=1)

        return final_output



# **예측**

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
cnn_model = getResnetModel().to(device)
cnn_model.load_state_dict(torch.load(model_path))
cnn_model.eval()

rnn_model = Classifier(hp.num_classes).to(device)
rnn_model.load_state_dict(torch.load(rnnmodel_path))
rnn_model.eval()

combined_model = CombinedModel(cnn_model, rnn_model, hp.num_classes).to(device)
combined_model.load_state_dict(torch.load(combined_model_path))
combined_model.eval()


<ipython-input-39-0298b9017e68>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn_model.load_state_dict(torch.load(model_path))
<ipython-input-39-0298b9017e68>:6: FutureW

CombinedModel(
  (cnn_model): ResNet(
    (conv1): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (conv2_x): Sequential(
      (0): BottleNeck(
        (residual_function): Sequential(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU()
          (6): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track

In [40]:
def top_k_accuracy(output, target, k=1):
    with torch.no_grad():
        topk = output.topk(k, dim=1).indices
        correct = topk.eq(target.view(-1, 1).expand_as(topk))
        return correct.sum().item()

In [41]:
def get_label(indices):
    final_label = []
    for index in indices:
        final_label.append(_label[index])
    return final_label

In [42]:
import matplotlib.pyplot as plt

probability_top1, probability_top5 = 0, 0
cnn_top1, cnn_top5 = 0, 0
rnn_top1, rnn_top5 = 0, 0
combined_top1, combined_top5 = 0, 0

with torch.no_grad():
        for i, (cnn, sequence, labels) in enumerate(testloader):
            cnn_data, sequence, target = cnn.float().to(device), sequence.to(device), labels.to(device)

            cnn_output = cnn_model(cnn_data)
            cnn_probs = F.softmax(cnn_output, dim=1)

            rnn_output = rnn_model(sequence)
            rnn_probs = F.softmax(rnn_output, dim=1)

            cnn_output_prob = cnn_probs * cnn_output
            rnn_output_prob = rnn_probs * rnn_output
            probability_output = cnn_output_prob + rnn_output_prob

            combined_output = combined_model(cnn_data, sequence)

            combined_top1 += top_k_accuracy(combined_output, target, k=1)
            combined_top5 += top_k_accuracy(combined_output, target, k=5)

            probability_top1 += top_k_accuracy(probability_output, target, k=1)
            probability_top5 += top_k_accuracy(probability_output, target, k=5)

            cnn_top1 += top_k_accuracy(cnn_output, target, k=1)
            cnn_top5 += top_k_accuracy(cnn_output, target, k=5)

            rnn_top1 += top_k_accuracy(rnn_output, target, k=1)
            rnn_top5 += top_k_accuracy(rnn_output, target, k=5)

    # if i % 10 == 0:
    #   plt.imshow(cnn_data[0, 0].to('cpu'), cmap='gray')
    #   plt.title(_label[labels[i]])
    #   plt.show()

    #   _, cnn_index = cnn_output.topk(5, dim=1)
    #   _, rnn_index = rnn_output.topk(5, dim=1)
    #   _, probability_index = probability_output.topk(5, dim=1)

    #   print(f"Cnn : {get_label(cnn_index[0])}")
    #   print(f"RNN : {get_label(rnn_index[0])}")
    #   print(f"Probability : {get_label(probability_index[0])}")


total_samples = len(testloader)*64

probability_top1_acc = probability_top1 / total_samples * 100
probability_top5_acc = probability_top5 / total_samples * 100

print(total_samples)
print(f"Probability Model Top-1 Accuracy: {probability_top1_acc:.2f}%")
print(f"Probability Model Top-5 Accuracy: {probability_top5_acc:.2f}%")
print(f"CNN Model Top-1 Accuracy: {cnn_top1 / total_samples * 100:.2f}%")
print(f"CNN Model Top-5 Accuracy: {cnn_top5 / total_samples * 100:.2f}%")
print(f"RNN Model Top-1 Accuracy: {rnn_top1 / total_samples * 100:.2f}%")
print(f"RNN Model Top-5 Accuracy: {rnn_top5 / total_samples * 100:.2f}%")
print(f"Combined Model Top-1 Accuracy: {combined_top1 / total_samples * 100:.2f}%")
print(f"Combined Model Top-5 Accuracy: {combined_top5 / total_samples * 100:.2f}%")

<ipython-input-29-233328b2b5c7>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequences_padded = pad_sequence([torch.tensor(seq, dtype=torch.float32) for seq in sequences],


310528
Probability Model Top-1 Accuracy: 79.16%
Probability Model Top-5 Accuracy: 96.17%
CNN Model Top-1 Accuracy: 76.79%
CNN Model Top-5 Accuracy: 95.38%
RNN Model Top-1 Accuracy: 71.16%
RNN Model Top-5 Accuracy: 93.21%
Combined Model Top-1 Accuracy: 73.55%
Combined Model Top-5 Accuracy: 94.26%
